In [5]:
import os
# to use or not to use GPU
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
!sudo pip install tensorflow --upgrade
# packages for model graph visualization
#!pip install -q pydot
# install graphviz https://graphviz.gitlab.io/download/ 

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub
!pip install -q -U tensorflow-text
import tensorflow_text as text
!pip install -q tf-models-official

import json
import re
import random
import numpy as np
import matplotlib.pyplot as plt

#save_path = "/content/drive/My Drive/Colab Notebooks/"

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
save_path = "/content/drive/My Drive/Colab Notebooks/"

In [8]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [9]:
# checking if we have access to a GPU

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
#@title Data Loading & Preprocessing


In [57]:
with open(save_path+'kafka_data_preprocessed_high_occurrence_50.json') as f:
    data = json.load(f)
    # shuffle data
    random.shuffle(data)
    # create description and assignee list
    desc_data = []
    assignee_data = []
    for item in data:
        desc_data.append(item['description'])
        assignee_data.append(item['assignee'])

In [58]:
print('sample assignees:', assignee_data[0:2])
print('sample descriptions:', desc_data[0:2])

sample assignees: ['Jay Kreps', 'Jiangjie Qin']
sample descriptions: ['Currently it is a bit of a mess:jkreps-mn:kafka-git jkreps$ ls binkafka-console-consumer-log4j.properties kafka-producer-perf-test.sh kafka-server-stop.sh zookeeper-server-stop.shkafka-console-consumer.sh kafka-producer-shell.sh kafka-simple-consumer-perf-test.sh zookeeper-shell.shkafka-console-producer.sh kafka-replay-log-producer.sh kafka-simple-consumer-shell.shkafka-consumer-perf-test.sh kafka-run-class.sh run-rat.shkafka-consumer-shell.sh kafka-server-start.sh zookeeper-server-start.shI think all the *-shell.sh scripts and all the *-simple-perf-test.sh scripts should die. If anyone has a use for these test classes we can keep them around and use the via kafka-run-class, but they are clearly not made for normal people to use. The *-shell.sh scripts are obsolete now that we have the *-console-*.sh scripts, since these do everything the old scripts did and more. I recommend we also delete the code for these.I woul

In [59]:
# transform description data to tensorflow dataset
descriptions = tf.data.Dataset.from_tensor_slices(desc_data)
for input in descriptions.take(2):
    print(input)

tf.Tensor(b'Currently it is a bit of a mess:jkreps-mn:kafka-git jkreps$ ls binkafka-console-consumer-log4j.properties kafka-producer-perf-test.sh kafka-server-stop.sh zookeeper-server-stop.shkafka-console-consumer.sh kafka-producer-shell.sh kafka-simple-consumer-perf-test.sh zookeeper-shell.shkafka-console-producer.sh kafka-replay-log-producer.sh kafka-simple-consumer-shell.shkafka-consumer-perf-test.sh kafka-run-class.sh run-rat.shkafka-consumer-shell.sh kafka-server-start.sh zookeeper-server-start.shI think all the *-shell.sh scripts and all the *-simple-perf-test.sh scripts should die. If anyone has a use for these test classes we can keep them around and use the via kafka-run-class, but they are clearly not made for normal people to use. The *-shell.sh scripts are obsolete now that we have the *-console-*.sh scripts, since these do everything the old scripts did and more. I recommend we also delete the code for these.I would like to change each tool so that it produces a usage line

In [60]:
# target vectorize assignee data
assignee_dict = {assignee: i for i, assignee in enumerate(list(set(assignee_data)))}

In [61]:
# create target vector
assignee_vector = [assignee_dict[assignee] for assignee in assignee_data]
assignee_vector[:10]

[0, 2, 28, 21, 21, 23, 8, 21, 30, 31]

In [62]:
# assignee target vector to one hot vector
assignee_tensor = tf.data.Dataset.from_tensor_slices(assignee_vector)

for assignee in assignee_tensor.take(2):
    print(assignee)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)


In [63]:
#@title BOW


In [64]:
input_vectorizer = layers.TextVectorization(
    split = "whitespace",
    output_mode='multi_hot')

In [65]:
input_vectorizer.adapt(descriptions)

In [66]:
output_vectorizer = layers.CategoryEncoding(num_tokens = len(assignee_dict), output_mode="one_hot")

In [67]:
for description in descriptions.take(2):
    print(description)
    print(input_vectorizer(description))

tf.Tensor(b'Currently it is a bit of a mess:jkreps-mn:kafka-git jkreps$ ls binkafka-console-consumer-log4j.properties kafka-producer-perf-test.sh kafka-server-stop.sh zookeeper-server-stop.shkafka-console-consumer.sh kafka-producer-shell.sh kafka-simple-consumer-perf-test.sh zookeeper-shell.shkafka-console-producer.sh kafka-replay-log-producer.sh kafka-simple-consumer-shell.shkafka-consumer-perf-test.sh kafka-run-class.sh run-rat.shkafka-consumer-shell.sh kafka-server-start.sh zookeeper-server-start.shI think all the *-shell.sh scripts and all the *-simple-perf-test.sh scripts should die. If anyone has a use for these test classes we can keep them around and use the via kafka-run-class, but they are clearly not made for normal people to use. The *-shell.sh scripts are obsolete now that we have the *-console-*.sh scripts, since these do everything the old scripts did and more. I recommend we also delete the code for these.I would like to change each tool so that it produces a usage line

In [68]:
for bow_assignee in assignee_tensor.take(2):
    print(bow_assignee)
    print(output_vectorizer(bow_assignee))

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)


In [69]:
bow_dataset = tf.data.Dataset.zip((descriptions, assignee_tensor))

def vectorize_dataset(description, assignee):

  return input_vectorizer(description), output_vectorizer(assignee)

bow_dataset = bow_dataset.map(vectorize_dataset)

for description, assignee in bow_dataset.take(2):
    print(description, assignee)

tf.Tensor([0. 1. 0. ... 0. 0. 0.], shape=(42790,), dtype=float32) tf.Tensor(
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor([0. 1. 0. ... 0. 0. 0.], shape=(42790,), dtype=float32) tf.Tensor(
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)


In [70]:
bow_dataset = bow_dataset.shuffle(len(list(bow_dataset)), seed = 42)

In [71]:
# batching the dataset
BATCH_SIZE = 32
VAL_SIZE = 0.15
bow_dataset = bow_dataset.batch(batch_size=BATCH_SIZE)

for line, label in bow_dataset.take(2):
    print(line.shape)
    
print(len(list(bow_dataset)))

(32, 42790)
(32, 42790)
136


In [72]:
# splitting the dataset into validation and test data
val_size = round(VAL_SIZE*(len(list(bow_dataset))))

val_dataset = bow_dataset.take(val_size) 
train_dataset = bow_dataset.skip(val_size)

In [73]:
len(list(train_dataset))

116

In [74]:
vocab_size = input_vectorizer.vocabulary_size()
bow_model = tf.keras.Sequential([
  tf.keras.layers.Dense(1024, input_shape=(vocab_size,), activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(len(assignee_dict), activation = "softmax")
])

In [75]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [76]:
bow_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [77]:
bow_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1024)              43817984  
                                                                 
 dense_7 (Dense)             (None, 256)               262400    
                                                                 
 dense_8 (Dense)             (None, 64)                16448     
                                                                 
 dense_9 (Dense)             (None, 36)                2340      
                                                                 
Total params: 44,099,172
Trainable params: 44,099,172
Non-trainable params: 0
_________________________________________________________________


In [78]:
for inputs,targets in train_dataset.take(1):
    print(inputs)
    outputs = bow_model(inputs)

    print(outputs.shape)
    print(outputs)
    print(targets)

tf.Tensor(
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]], shape=(32, 42790), dtype=float32)
(32, 36)
tf.Tensor(
[[0.02692838 0.02727757 0.02934074 ... 0.02670073 0.03042019 0.02790688]
 [0.02760849 0.02778633 0.02817575 ... 0.02769669 0.02764769 0.02770082]
 [0.0279574  0.02816743 0.02760368 ... 0.02774283 0.02775576 0.02766822]
 ...
 [0.02736332 0.02817666 0.02850916 ... 0.02725299 0.02842584 0.02827456]
 [0.02730815 0.02779472 0.02886209 ... 0.02752302 0.02877241 0.0276683 ]
 [0.02800682 0.02746277 0.02797613 ... 0.02696937 0.02936696 0.02778163]], shape=(32, 36), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(32, 36), dtype=float32)


In [ ]:
bow_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1096: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  api_dispatcher = getattr(op_dispatch_handler, TYPE_BASED_DISPATCH_ATTR,


116/116 [==============================] - ETA: 0s - loss: 3.2592 - accuracy: 0.1512WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7f5253fb4320> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
116/116 [==============================] - 9s 51ms/step - loss: 3.2592 - accuracy